Download the github HTML files, upload them to your drive, and find the paths to them - it works just like a regular file, start the path after gdrive/My Drive/. (Not web scraping quite yet because the program is prone to mess up if the website data changes).

In [0]:
path = 'gdrive/My Drive/ML/Projects/Underlords Bot/under.html'
path2 = 'gdrive/My Drive/ML/Projects/Underlords Bot/stats.html'

Only the first 3 cells are working so far (assuming the client has the required file and path, which will be uploaded to the github asap). 

The other cells are just for testing algorithms but they have not been completed yet and are in no working order. 

The first 3 cells scrape the data on underlords characters and their synergies / health / damage etc. It contains whatever info needed (and probably some excess info based on how we use the data)

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
from bs4 import BeautifulSoup
from collections import defaultdict
from operator import itemgetter
import numpy as np
import tensorflow
import itertools
from copy import deepcopy

In [0]:

fr = open(path, 'r', encoding='utf8').read()
soup = BeautifulSoup(fr, "html.parser")


tags = soup('div class')
tiers = soup('u')
tags = soup('li')
container = []
for tag in tags:
    if not tag:
        continue
    info = list(filter(lambda st: st != '' and '<' not in st, str(tag).split()))
    if len(info) <= 5:
        extract = str(tag.contents[0]).strip()
        container.append(extract)
        if str(tag.contents[0]).strip() == 'Lich':
            break


synergies = dict()
levels = dict()

synergies['S Tier'] = container[:3]
synergies['A Tier'] = container[3:10]
synergies['B Tier'] = container[10:15]
synergies['C Tier'] = container[15:23]
levels['1'] = container[23:37]
levels['2'] = container[37:51]
levels['3'] = container[51:66]
levels['4'] = container[66:78]
levels['5'] = container[78:83]

fr = open(path2, 'r', encoding='utf8').read()
soup = BeautifulSoup(fr, "html.parser")

'''
tags = soup('u')
for tag in tags:
    print(tag.contents)'''


soup = BeautifulSoup(fr, "html.parser")
tags = soup('li') 
st = ""

collection = []
for tag in tags:
    if tag != None and str(tag.contents[0]).startswith('<strong>'):
        extract = ["Alliance 1", "Alliance 2", 'Alliance 3', 'Alliance 4', 'Health', 'DPS', 'Attack Range', 'Armor']
        
        for e in extract:
            
            if str(tag.contents[0]).find(e) != -1:
                collection.append(str(tag.contents[1]).strip())
                
            
            

info = {}
tags = soup('u')
i = 0
for tag in tags:
    if str(tag)[3:-4] == 'Puck' or str(tag)[3:-4] == 'Dragon Knight' or str(tag)[3:-4] == 'Lycan':
        info[str(tag)[3:-4]] = collection[i:i+8]
        i += 8
    else: 
        info[str(tag)[3:-4]] = collection[i:i+7]
        i += 7



l1 = []
for nm in levels:
    for si in levels[nm]:
        l1 += [si]

l2 = []
for key in l1:
    if 'Wind' in key:
        l2.append(key)
    if 'Nature' in key:
        l2.append(key)

st = ''
for ao in info:
    if 'Nature' in ao:
         st = ao
        
for m in l2:
    if 'Wind' in m:
        info[m] = info['Wind Ranger']
    else:
        info[m] = info[ao]

# give each hero a number for q-table access later on 
heroes_to_num = {}
for index, hero in enumerate(info.keys()): 
    heroes_to_num[hero.strip()] = index

num_to_heroes = {}
for index, hero in enumerate(info.keys()): 
    num_to_heroes[index] = hero

i = 0
synergies_to_num = {}
all_synergies = []

for tier in synergies:
    for synergy in synergies[tier]:
        synergies_to_num[synergy] = i
        all_synergies.append(synergies)
        i += 1

'''
add_syns = []
for key in synergies_to_num:
    add_syns.append(key[:-1])
for syn in add_syns:
    synergies_to_num[syn] = synergies_to_num[key]'''

from difflib import SequenceMatcher

# given a synergy which may not be in the dictionary, this tries to find the closest matching one
def similarity_function(access):
    # return the synergy string of which access most closely correlates to
    max_sim_synergy = ''
    max_ratio = 0

    for key in synergies_to_num:
        if SequenceMatcher(None, access, key).ratio() > max_ratio:
            max_ratio = SequenceMatcher(None, access, key).ratio()
            max_sim_synergy = key
        return max_sim_synergy

In [0]:
levels

{'1': ['Drow Ranger',
  'Axe',
  'Tinker',
  'Batrider',
  'Tusk',
  'Enchantress',
  'Tiny',
  'Clockwerk',
  'Warlock',
  'Bloodseeker',
  'Bounty Hunter',
  'Shadow Shaman',
  'Anti-Mage',
  'Ogre Magi'],
 '2': ['Witch Doctor',
  'Chaos Knight',
  'Queen of Pain',
  'Pudge',
  'Crystal Maiden',
  'Treant Protector',
  'Beastmaster',
  'Timbersaw',
  'Juggernaut',
  'Slardar',
  'Natures Prophet',
  'Luna',
  'Puck',
  'Morphling'],
 '3': ['Shadow Fiend',
  'Slark',
  'Viper',
  'Lycan',
  'Abaddon',
  'Razor',
  'Sand King',
  'Sniper',
  'Windranger',
  'Arc Warden',
  'Venomancer',
  'Lina',
  'Phantom Assassin',
  'Terrorblade',
  'Omniknight'],
 '4': ['Kunkka',
  'Doom',
  'Medusa',
  'Lone Druid',
  'Keeper of the Light',
  'Troll Warlord',
  'Mirana',
  'Necrophos',
  'Dragon Knight',
  'Disruptor',
  'Alchemist',
  'Templar Assassin'],
 '5': ['Tidehunter', 'Enigma', 'Techies', 'Gyrocopter', 'Lich']}

In our game environment, we create an observation space with factors like gold, level, # of synergies, # of heroes, etc. Each synergy and hero will have its own q-value. 

The q-value for a particular synergy will take into account how good the synergy is, how many heroes needed per level, etc. when backpropagated. 

In [0]:
lr = 0.1
discount = 0.95
episodes = 5000

# (level 1, level 2, level 3), synergies, level
q_table = np.random.uniform(low=-2,high=0, size=(3, 9, 10)) 

In [0]:
synergies_to_hero = defaultdict(list)
for hero in info:
  synergy1 = similarity_function(info[hero][0])
  synergy2 = similarity_function(info[hero][1])
  synergies_to_hero[synergies_to_num[synergy1]] += [hero]
  synergies_to_hero[synergies_to_num[synergy2]] += [hero]

Variables:

synergies_to_num - encodes synergy 

info - maps hero names to stats such as synergies (some are inaccurate, so use similarity function)

levels - maps level / tier to a list of characters

hero_probs - maps hero encoding to probability of getting hero

synergies_to_hero - key will be synergy (encoded?), and will map to hero encodings 

syns - encoded keys to count

Each hero has some probability based on the tier, and they have synergies 


In [0]:
hero_probs = defaultdict(list) # probability of getting a hero 


# action will be a list of moves (strings), inventory is a dictionary of hero encodings to count
def expected_value(actions, level, inventory, syns={}):
  expected = 0

  # if not in inventory (inventory[piece] = 0), then count the synergy as new
  for action in actions:
    if action != 'sell':
      if inventory[pieces_to_num[action]] == 0:
        synergy1 = similarity_function(info[action][0])
        synergy2 = similarity_function(info[action][1])
        syns[synergies_to_num[synergy1]] += 1 
        syns[synergies_to_num[synergy2]] += 1 
      inventory[pieces_to_num[action]] += 1

  
  for hero in info:
    hero_probs[pieces_to_num[hero]] = prob(hero) # returns probability of getting that particular hero, based on lvl

  # expected value of getting sets of pieces which gets this synergy boost
  for syn in syns:
    # find next closest # of synergies needed for boost for each of the synergies 
    next = next_closest(syn)
    # find all combination of heroes which make it possible to get synergy boost
    if next != 3:
      for comb in itertools.combinations(synergies_to_hero[syn], next):
        f = True
        for item in comb:
          if inventory[item] != 0: # probability of affecting synergy = 0
            f = False
        if not f: 
          continue
        expected += 50 * (hero_probs[comb[0]] * hero_probs[comb[1]])
  # probability of getting sets of pieces (or) which will get this synergy boost 
  # probability of leveling up the pieces already in inventory if level up
  # probability of leveling up pieces in shop if deciding to get them

  # recursive up


In [0]:
# dictionary of piece # to synergy

for round in range(5):
  # generate troops
  generate_troops()
  
  # expected value in 5 rounds using a recursive function and

Perhaps we can choose randomly from the action space size instead of 

In [0]:
synergies_to_num

In [0]:
# tweak these
SECOND_LVL_REWARD  = 60
THIRD_LVL_REWARD = 180
FIRST_SYN_REWARD = 50
SECOND_SYN_REWARD = 130
THIRD_SYN_REWARD = 200

OBSERVATION_SPACE_VALUES = [80, 10]
# each of these cells will contain count of a particular hero
for p in range(60):
  OBSERVATION_SPACE_VALUES.append(1)
# will contain the number of each synergy
for p in range(23):
  OBSERVATION_SPACE_VALUES.append(1)
OBSERVATION_SPACE_VALUES = tuple(OBSERVATION_SPACE_VALUES)

# OBSERVATION_SPACE_VALUES = (80, 10 ()) # 23, 60) # gold, level, # of synergies, # of heroes, 

ACTION_SPACE_SIZE = 60 # each of the heroes or saving or ...
class GameEnv:

  
  def reset(self):
    self.pieces_to_num = heroes_to_num
    self.num_to_pieces = num_to_heroes
    self.synergies_to_num = synergies_to_num
    self.piece_stats = info
    self.gold = 1
    self.lev = 1
    self.xp = 1
    self.inventory = dict()
    self.iter = 0
    for value in heroes_to_num.values(): 
        self.inventory[value] = [0, 0, 0] # number of level 1, 2, and 3 troops
    self.exp_needed = [2, 4, 8, 12, 16, 20, 24, 28, 32, 36, 40] # experience needed to level up
    self.synergy_inventory = dict()
    for n in range(i):
        self.synergy_inventory[n] = 0
    self.existing = {}
    self.rnd = 1
    self.episode_step = 0
  
  
  def __init__(self):
    self.pieces_to_num = heroes_to_num
    self.num_to_pieces = num_to_heroes
    self.synergies_to_num = synergies_to_num
    self.piece_stats = info
    self.gold = 1
    self.lev = 1
    self.xp = 1
    self.inventory = dict()
    self.iter = 0
    for value in heroes_to_num.values(): 
        self.inventory[value] = [0, 0, 0] # number of level 1, 2, and 3 troops
    self.exp_needed = [2, 4, 8, 12, 16, 20, 24, 28, 32, 36, 40] # experience needed to level up
    self.synergy_inventory = dict()
    for n in range(i):
        self.synergy_inventory[n] = 0
    self.existing = {}
    self.rnd = 1
    self.episode_step = 0
  
  # updates level of pieces on board if there are enough
  def update_level(piece, index):
    if self.inventory[self.pieces_to_num[piece]][index] == 3:
      self.inventory[self.pieces_to_num[piece]][index] = 0
      self.inventory[self.pieces_to_num[piece]][index + 1] += 1
  
  # choice is the action number
  def action(choice):
    # if a hero is bought
    if 0 <= choice <= 59: 
      self.inventory[self.choice][0] += 1
      
      # take care of merging / levelling
      update_level(self.num_to_pieces[choice], 0)
      update_level(self.num_to_pieces[choice], 1)
      
      # count them as synergy
      if self.inventory[self.choice] == [1, 0, 0]:
        first_syn = similarity_function[info[choice][0]]
        second_syn = similarity_function[info[choice][1]]
        self.synergy_inventory[first_syn] += 1
        self.synergy_inventory[second_syn] += 1
        
      
      # take away gold if they bought a piece
      for key in levels:
        if self.num_to_pieces[choice] in levels[key]:
          self.gold -= int(key)
      
      
      
  def step(self, action):
    self.episode_step += 1
    reward = 0 
    
    for piece in self.inventory:
      if self.inventory[piece][1] % 1 == 0: # level 2  piece
        reward += (SECOND_LVL_REWARD * self.inventory[piece][1])
      elif self.inventory[piece][2] == 1:
        reward += THIRD_LVL_REWARD
    
    for synergy in self.synergy_inventory:
      if self.synergy_inventory[synergy] == 3:
        reward += FIRST_SYN_REWARD
        
        
    reward += self.gold * 2
    
    obs = [self.gold, self.lev]
    
    for m in range(60):
      obs.append(0)
    for m in range(23):
      obs.append(0)
    
    for piece in self.inventory:
      obs[2 + self.piece_to_num[piece]] = sum(self.inventory[piece])
      
    for synergy in self.synergy_inventory:
      obs[62 + self.synergies_to_num[synergy]] = self.synergy_inventory[synergy]
      
    
    if self.episode_step >= 35:
      done = True
      
    return obs, reward, done

We're making our own custom reinforcement learning algorithm. Basically, using a replay memory, we will average the sum of rewards over the next few rounds which is a result of purchasing a particular piece. We will then create a model which attempts to predict the sum of rewards given the state and available pieces

In [0]:
agent_model = Sequential()
agent_model.add(Dense(100, activation='relu'))
agent_model.add(Dense(60, activation='linear'))

target_model = Sequential()
target_model.add(Dense(100, activation='relu'))
target_model.add(Dense(60, activation='linear'))

env = GameEnv()

for episode in range(35000):
  generate_troops() # list of troops
  agent_model.predict()

In [0]:
env = GameEnv()

In [0]:


class DQNAgent:
    def create_model(self):
        model = Sequential()

        #model.add(Conv2D(256,  (3, 3), input_shape=OBSERVATION_SPACE_VALUES)) # observation space values = .x.x... 
        #model.add(Activation('relu'))
        #model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dense(1024)) # fully connected layer
        model.add(Dropout(0.2))

        # model.add(Conv2D(256, (3, 3)))
        model.add(Activation('relu'))
        #model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.2))

        model.add(Flatten()) # 1D feature vectors
        model.add(Dense(64)) # fully connected layer

        model.add(Dense(ACTION_SPACE_SIZE, activation='linear')) # action_space_Size = # of choices
        model.compile(loss='mse', optimizer=Adam(lr=0.001), metrics=['accuracy'])
        return model

    def __init__(self):
        # main network
        self.model = self.create_model()

        # the model that we query for future q-models will be more stable than the one being fitted each step
        # this means that there is more consistnecy while training (if the weights fluctuate, q-values won't be on the same scale)
        # target network - initially same as main network
        self.target_model = self.create_model()
        self.target_model.set_weights(self.model.get_weights())

        # array w/ last n steps for training - also prevents fluctuations
        self.replay_memory = deque(maxlen=REPLAY_MEMORY_SIZE)


        self.target_update_counter = 0 # when to update target network w/ main network's weights
    
    # Adds step's data to a memory replay array / update replay memory
    # (observation space, action, reward, new observation space, done)
    def update_replay_memory(self, transition):
        self.replay_memory.append(transition)

    # queries the main neural network for q-values given the observation space
    def get_qs(self, state):
        return self.model.predict(np.array(state).reshape(-1, *state.shape) / 255)

    
    # trains network every step during episode
    def train(self, terminal_state, step):
        # only start training if we have at least a specified # of samples
        if len(self.replay_memory) < MIN_REPLAY_MEMORY_SIZE:
            return

        # separate states and their q-values from that minibatch of memory, including the new state and the future qs of future states
        
        # get a minibatch of random samples from memory replay table
        minibatch = random.sample(self.replay_memory, MINIBATCH_SIZE)

        # get current states and query model for q-values
        current_states = np.array(transition for transition in minibatch) / 255
        current_qs_list = self.model.predict(current_states)

        # get future states from minibatch, then query NN for q-values
        # when using target network, query it - else use main network
        # transition[3] is the new observation space
        new_current_states = np.array([transition[3] for transition in minibatch]) / 255
        future_qs_list = self.target_model.predict(new_current_states)
    
        X = []
        y = []

        # for each observation in the minibatch
        for index, (current_state, action, reward, new_current_state, done) in enumerate(minibatch):
            # if not a terminal state (where the reward is then backpropagated), get new q from future states
            # almost like q-learning except use only part of the equation
            if not done:
                max_future_q = np.max(future_qs_list[index])
                new_q = reward + DISCOUNT * max_future_q
            else:
                new_q = reward

            # update q vaues for given state
            current_qs = current_qs_list[index]
            current_qs[action] = new_q

            # add current state, and current q 
            X.append(current_state)
            y.append(current_qs)
        # fit on all samples as one batch - log only on terminal state
        self.model.fit(np.array(X) / 255, np.array(y), batch_size=MINIBATCH_SIZE, verbose=0
                        , shuffle=False) 

        # Update target network counter every end of episode
        '''
        Note that the terminal state is not related to the replay memory - the model is trained every single step
        and if the step passed in to this method is the final step / end of episode, then the target model weights are updated
        '''
        if terminal_state:
            self.target_update_counter += 1
        
        # if counter reaches et value, update target network with weights
        if self.target_update_counter > UPDATE_TARGET_EVERY:
            self.target_model.set_weights(self.model.get_weights)
            self.target_update_counter = 0  

In [0]:
timesteps = 35
input_features = 85
output_features = 60

model = Sequential()
# Input()
# SimpleRNN input: (batch_size, timesteps, input_features)
model.add(SimpleRNN(output_features, return_sequences=True)) 
model.add(SimpleRNN(output_features, return_sequences=True))
model.add(SimpleRNN(output_features, return_sequences=True))
model.add(SimpleRNN(output_features))
model.add(Dense(1, activation='softmax'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
# input train contains the state / features
history = model.fit(input_train, y_train, epochs=1)

inputs = ... # shape of (timesteps, input_features)


In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Activation, Flatten
from keras.optimizers import Adam
from collections import deque

REPLAY_MEMORY_SIZE = 50_000

epsilon = 0.8

agent = DQNAgent()

for episode in range(30000):

    # restarting episode - reset episode reward and step
    episode_reward = 0
    step = 1

    # reset env and get current state
    current_state = env.reset()

    # reset flag and iterate until end of episode
    done = False
    while not done:
        # query a model for q-values rather than table
        if np.random.random() > epsilon:
            action = np.argmax(agent.get_qs(current_state))
        else:
            action = np.random.randint(0, ACTION_SPACE_SIZE)
        
        new_state, reward, done = env.step(action)

        # transform continuous state to discrete and count reward
        episode_reward += reward

        # update replay memory and train main network
        agent.update_replay_memory((current_state, action, reward, new_state, done))
        agent.train(done, step)
        current_state = new_state
        step += 1

# now we have to train


We need a q-table to store the values 